In [ ]:
!pip install pandas nltk beautifulsoup4 datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is i

In [ ]:
KEYWORDS = [
    'silver', 'silver prices', 'silver price', 'silver futures',
    'spot silver', 'silver market', 'silver demand', 'silver supply',
    'silver production', 'silver mining', 'silver bullion', 'silver bars',
    'silver coins', 'silver reserves', 'silver exports', 'silver imports',
    'silver ETF', 'precious metals', 'London Bullion Market', 'COMEX silver'
]


In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("danidanou/Bloomberg_Financial_News", split="train")
df = pd.DataFrame(dataset)

# Combine Headline and Article
df['text'] = df['Headline'].fillna('') + '. ' + df['Article'].fillna('')

# Clean HTML tags
def clean_html(raw_html):
    return BeautifulSoup(raw_html, "html.parser").get_text()

df['text'] = df['text'].apply(clean_html)

# Define related keywords
pattern = re.compile('|'.join(KEYWORDS), flags=re.IGNORECASE)

# Filter for keyword-related articles
df['is_keyword_related'] = df['text'].apply(lambda x: bool(pattern.search(x)))
keyword_df = df[df['is_keyword_related']].copy()

# Keep only relevant columns
keyword_df = keyword_df[['Date', 'text']].drop_duplicates().dropna()
keyword_df['Date'] = pd.to_datetime(keyword_df['Date'], errors='coerce')
keyword_df = keyword_df.dropna(subset=['Date']).reset_index(drop=True)

# Save to file
keyword_df.to_csv('articles_filtered.csv', index=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.00k [00:00<?, ?B/s]

bloomberg_financial_data.parquet.gzip:   0%|          | 0.00/482M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/446762 [00:00<?, ? examples/s]